# Movie Chatbot with Azure Cosmos DB for NoSQL

In this sample, we'll demonstrate how to build a RAG Pattern application using a subset of the Movie Lens dataset. This sample will leverage the Python SDK for Azure Cosmos DB for NoSQL to perform vector search for RAG, store and retrieve chat history, and store the vectors of the chat history to use as a semantic cache. Azure OpenAI to generate embeddings and LLM completions.

At the end we will create a simple UX using Gradio to allow users to type in questions and display responses generated by a GPT model or served from the cache. The resopnses will also display an elapsed time so you can see the impact caching has on performance versus generating a response.

**Important Note**: This sample requires you to have a Azure Cosmos DB for NoSQL account setup. To get started, visit:

- [Azure Cosmos DB for NoSQL Python Quickstart](https://learn.microsoft.com/azure/cosmos-db/nosql/quickstart-python?pivots=devcontainer-codespace)
- [Azure Cosmos DB for NoSQL Vector Search](https://learn.microsoft.com/azure/cosmos-db/nosql/vector-search)


# Preliminaries <a class="anchor" id="preliminaries"></a>

First, let's start by installing the packages that we'll need later.


In [1]:
! /usr/bin/python3 -m pip install openai pymongo python-dotenv urlopen azure-cosmos tenacity aiohttp gradio > /dev/null

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Use example.env as a template to provide the necessary keys, endpoints, and parameters in your own .env file.


In [2]:
# Import the required libraries
import time
import json
import uuid
from dotenv import load_dotenv
from openai import AzureOpenAI
import gradio as gr
from azure.cosmos import CosmosClient
import openai
from azure.identity import (
    DefaultAzureCredential,
    get_bearer_token_provider,
    AzureDeveloperCliCredential,
)
import os

load_dotenv()

COSMOS_NOSQL_URL = os.getenv("COSMOS_NOSQL_URL")
COSMOS_NOSQL_KEY = os.getenv("COSMOS_NOSQL_KEY")
COSMOS_NOSQL_DATABASE_NAME = os.getenv("COSMOS_NOSQL_DATABASE_NAME")
COSMOS_NOSQL_COLLECTION_NAME = os.getenv("COSMOS_NOSQL_COLLECTION_NAME")
COSMOS_NOSQL_VECTOR_PROPERTY_NAME = os.getenv("COSMOS_NOSQL_VECTOR_PROPERTY_NAME")
COSMOS_NOSQL_CACHE_COLLECTION_NAME = os.getenv("COSMOS_NOSQL_CACHE_COLLECTION_NAME")

AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Create the Azure Cosmos DB for NoSQL client
cosmos_client = CosmosClient(url=COSMOS_NOSQL_URL, credential=COSMOS_NOSQL_KEY)

# Create the OpenAI client
azure_credential = AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))
token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)
openai_client = openai.AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider,
)

/Users/prateek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/prateek/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Database and collections

Please make sure that you have the movies and the cache containers setup already. To know more about how to setup vector search enabled containers, please refer to the `setup.ipynb` notebook.


In [3]:
# Get the cosmos databases and containers to work with that we created during setup.
db = cosmos_client.get_database_client(COSMOS_NOSQL_DATABASE_NAME)
movies_container = db.get_container_client(COSMOS_NOSQL_COLLECTION_NAME)
cache_container = db.get_container_client(COSMOS_NOSQL_CACHE_COLLECTION_NAME)

# Generate embeddings from Azure OpenAI

This is used to vectorize the user input for the vector search.
**IMPORTANT:**

- If you use the sample MovieLens data as is, you'll need to use the text-3-embedding-large model with `dimensions=256` If the dimensionality specified below doesn't match this, an error will be thrown when you implement the chatbot as query vectors in vector search need to match dimension size of the data.
- Additionally, if you use a model other than `text-3-embedding-large`, these aren't compatible with the vector embeddings in the sample data provided and your search results won't be accurate. You should regenerate these vectors with your desired model.


In [ ]:
# generate openai embeddings
def generate_embeddings(text):
    """
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    """
    print("Generating embeddings for: ", text)
    print("Emedding model used: ", AZURE_OPENAI_ADA_DEPLOYMENT)
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_ADA_DEPLOYMENT,
    )
    embeddings = response.model_dump()
    return embeddings["data"][0]["embedding"]

In [11]:
generate_embeddings("test")

Generating embeddings for:  test
Emedding model used:  text-embedding-ada-002


[-0.0016663400456309319,
 -0.014076396822929382,
 0.0014772829599678516,
 -0.018416794016957283,
 -0.0073793800547719,
 0.01851527951657772,
 -0.009714893996715546,
 -0.02898288518190384,
 -0.006081481464207172,
 -0.014273367822170258,
 0.011487633921205997,
 0.007351241540163755,
 0.0013779180590063334,
 -0.01177605614066124,
 -0.007199995685368776,
 0.012901605106890202,
 0.0468791127204895,
 0.0006946747307665646,
 0.017319384962320328,
 -0.028701497241854668,
 0.010692714713513851,
 0.00711909681558609,
 -0.0035155818331986666,
 0.0012838292168453336,
 -0.03013657219707966,
 -0.014941662549972534,
 0.0017577909165993333,
 -0.037199392914772034,
 -0.0030794315971434116,
 0.002884219167754054,
 0.0190217774361372,
 -0.028814053162932396,
 -0.026619233191013336,
 -0.02991146221756935,
 -0.02341141737997532,
 -0.008568241260945797,
 -0.0037776236422359943,
 -0.014315575361251831,
 0.01788215897977352,
 -0.00023148496984504163,
 0.010404293425381184,
 0.013126714155077934,
 0.0057578864

# Vector Search in Azure Cosmos DB for NoSQL

This defines a function for performing a vector search over the movies data and chat cache collections. Function takes a collection reference, array of vector embeddings, and optional similarity score to filter for top matches and number of results to return to filter further.


In [14]:
# Perform a vector search on the Cosmos DB container
def vector_search(container, vectors, similarity_score=0.02, num_results=5):
    # Execute the query
    results = container.query_items(
        query="""
        SELECT TOP @num_results  c.overview, VectorDistance(c.vector, @embedding) as SimilarityScore 
        FROM c
        WHERE VectorDistance(c.vector,@embedding) > @similarity_score
        ORDER BY VectorDistance(c.vector,@embedding)
        """,
        parameters=[
            {"name": "@embedding", "value": vectors},
            {"name": "@num_results", "value": num_results},
            {"name": "@similarity_score", "value": similarity_score},
        ],
        enable_cross_partition_query=True,
        populate_query_metrics=True,
    )
    results = list(results)
    # Extract the necessary information from the results
    formatted_results = []
    for result in results:
        score = result.pop("SimilarityScore")
        formatted_result = {"SimilarityScore": score, "document": result}
        formatted_results.append(formatted_result)

    # #print(formatted_results)
    metrics_header = dict(container.client_connection.last_response_headers)
    # print(json.dumps(metrics_header,indent=4))
    return formatted_results

# Get recent chat history

This function provides conversational context to the LLM, allowing it to better have a conversation with the user.


In [15]:
def get_chat_history(container, completions=3):
    results = container.query_items(
        query="""
        SELECT TOP @completions *
        FROM c
        ORDER BY c._ts DESC
        """,
        parameters=[
            {"name": "@completions", "value": completions},
        ],
        enable_cross_partition_query=True,
    )
    results = list(results)
    return results

# Chat Completion Function

This function assembles all of the required data as a payload to send to a GPT model to generate a completion


In [ ]:
def generate_completion(user_prompt, vector_search_results, chat_history):

    system_prompt = """
    You are an intelligent assistant for movies. You are designed to provide helpful answers to user questions about movies in your database.
    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly. Greet with First Name if available.
    You will be provided with a user question, a list of relevant movies, and a chat history. Use this information to generate a response.
        - Only answer questions related to the information provided below. Provide at least 3 candidate movie answers in a list.
        - Write two lines of whitespace between each answer in the list.
    """

    # Create a list of messages as a payload to send to the OpenAI Completions API

    # system prompt
    messages = [{"role": "system", "content": system_prompt}]

    # chat history
    for chat in chat_history:
        messages.append(
            {"role": "user", "content": chat["prompt"] + " " + chat["completion"]}
        )

    # user prompt
    messages.append({"role": "user", "content": user_prompt})

    # vector search results
    for result in vector_search_results:
        messages.append({"role": "system", "content": json.dumps(result["document"])})

    print("Messages going to openai", messages)
    # Create the completion
    response = openai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME, messages=messages, temperature=0.1
    )
    return response.model_dump()


def chat_completion(history_container, movies_container, user_input):

    # Generate embeddings from the user input
    user_embeddings = generate_embeddings(user_input)
    # Query the chat history cache first to see if this question has been asked before
    cache_results = vector_search(
        container=history_container,
        vectors=user_embeddings,
        similarity_score=0.99,
        num_results=1,
    )

    if len(cache_results) > 0:  # CACHE HIT
        # If the cache has a result, return the cached completion
        print("\n Cached Result\n")
        return cache_results[0]["document"]["completion"]

    else:  # CACHE MISS
        # If the cache does not have a result, proceed with the vector search and completion generation

        # perform vector search on the movie collection
        print("\n New result\n")
        search_results = vector_search(movies_container, user_embeddings)

        print("Getting Chat History\n")
        # chat history
        chat_history = get_chat_history(history_container, 10)

        # generate the completion
        print("Generating completions \n")
        completions_results = generate_completion(
            user_input, search_results, chat_history
        )

        print("Caching response \n")
        # cache the response
        cache_response(
            history_container, user_input, user_embeddings, completions_results
        )

        print("\n")
        # Return the generated LLM completion
        return completions_results["choices"][0]["message"]["content"]

# Save Generated Responses as Chat History

Save the user prompts and generated completions in a conversation. Used to answer the same questions from other users. This is cheaper and faster than generating results each time.


In [35]:
# Save the chat document to the Cosmos DB container
# Which serves as a "Semantic cache" for the chat history


def save_cache(container, user_prompt, prompt_vectors, response):
    # Create a dictionary representing the chat document
    chat_document = {
        "id": str(uuid.uuid4()),
        "prompt": user_prompt,
        "completion": response["choices"][0]["message"]["content"],
        "completionTokens": str(response["usage"]["completion_tokens"]),
        "promptTokens": str(response["usage"]["prompt_tokens"]),
        "totalTokens": str(response["usage"]["total_tokens"]),
        "model": response["model"],
        "vector": prompt_vectors,
    }
    # Insert the chat document into the Cosmos DB container
    container.create_item(body=chat_document)
    print("item inserted into history.", chat_document)


# Perform a vector search on the Cosmos DB container
def get_cache(container, vectors, similarity_score=0.0, num_results=5):
    # Execute the query
    results = container.query_items(
        query="""
        SELECT TOP @num_results *
        FROM c
        WHERE VectorDistance(c.vector,@embedding) > @similarity_score
        ORDER BY VectorDistance(c.vector,@embedding)
        """,
        parameters=[
            {"name": "@embedding", "value": vectors},
            {"name": "@num_results", "value": num_results},
            {"name": "@similarity_score", "value": similarity_score},
        ],
        enable_cross_partition_query=True,
        populate_query_metrics=True,
    )
    results = list(results)
    print("Cached results: ", results)
    return results

# LLM Pipeline function

This function defines the pipeline for our RAG Pattern application.

1. When user submits a question, the cache is consulted first for an exact match (Cache hit).
2. If no match (Cache Miss) then a vector search is made
3. Chat history gathered,
4. The LLM generates a response, which is then cached before returning to the user.

<img src="./Images/agent-memory-in-action.png" alt="my image" width="800">


In [36]:
def chat_completion(chathistory_container, movies_container, user_input):
    print("starting completion")
    # Generate embeddings from the user input
    user_embeddings = generate_embeddings(user_input)
    # Query the chat history cache first to see if this question has been asked before
    cache_results = get_cache(
        container=chathistory_container,
        vectors=user_embeddings,
        similarity_score=0.99,
        num_results=1,
    )
    if len(cache_results) > 0:
        print("[CACHE HIT] Cached Result\n")
        return cache_results[0]["completion"], True

    else:
        # perform vector search on the movie collection
        print("[CACHE MISS] New result\n")
        search_results = vector_search(movies_container, user_embeddings)

        print("Getting Chat History\n")
        # chat history
        chat_history = get_chat_history(chathistory_container, 3)
        # generate the completion
        print("Generating completions \n")
        completions_results = generate_completion(
            user_input, search_results, chat_history
        )

        print("Caching response \n")
        # cache the response
        save_cache(
            chathistory_container, user_input, user_embeddings, completions_results
        )

        print("\n")
        # Return the generated LLM completion
        return completions_results["choices"][0]["message"]["content"], False

# Create a simple UX in Gradio


In [ ]:
chat_history = []
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Movie Assistant")

    msg = gr.Textbox(label="Ask me about movies in the Movie Database!")
    clear = gr.Button("Clear")

    def user(user_message, chat_history, pk):
        # Create a timer to measure the time it takes to complete the request
        start_time = time.time()
        # Get LLM completion
        response_payload, cached = chat_completion(
            cache_container, movies_container, user_message
        )
        # Stop the timer
        end_time = time.time()
        elapsed_time = round((end_time - start_time) * 1000, 2)
        response = response_payload
        print(response_payload)
        # Append user message and response to chat history
        details = f"\n (Time: {elapsed_time}ms)"
        if cached:
            details += " (Cached Hit)"
        chat_history.append([user_message, response_payload + details])

        return gr.update(value=""), chat_history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)

    clear.click(lambda: None, None, chatbot, queue=False)

/Users/prateek/Library/Python/3.9/lib/python/site-packages/gradio/utils.py:1002: UserWarning: Expected 3 arguments for function <function user at 0x15e892af0>, received 2.
  warnings.warn(
/Users/prateek/Library/Python/3.9/lib/python/site-packages/gradio/utils.py:1006: UserWarning: Expected at least 3 arguments for function <function user at 0x15e892af0>, received 2.
  warnings.warn(


In [38]:
# launch the gradio interface
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/prateek/Library/Python/3.9/lib/python/site-packages/gradio/helpers.py:978: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


starting completion
Generating embeddings for:  my favorite genre?
Emedding model used:  text-embedding-ada-002
Cached results:  []
[CACHE MISS] New result

Getting Chat History

Generating completions 

Messages going to openai [{'role': 'system', 'content': '\n    You are an intelligent assistant for movies. You are designed to provide helpful answers to user questions about movies in your database.\n    You are friendly, helpful, and informative and can be lighthearted. Be concise in your responses, but still friendly. Greet with First Name if available.\n    You will be provided with a user question, a list of relevant movies, and a chat history. Use this information to generate a response.\n        - Only answer questions related to the information provided below. Provide at least 3 candidate movie answers in a list.\n        - Write two lines of whitespace between each answer in the list.\n    '}, {'role': 'user', 'content': 'what isn my name? Your name is Prateek Singh! If you h

In [39]:
# be sure to run this cell to close or restart the gradio demo
demo.close()

Closing server running on port: 7860
